# Lab 3: Text Generation with an N-Gram Language Model

In this lab, you will implement two text generation strategies covered in lecture: Greedy Search and Sampling.

Here's a revision of the n-gram language model implementation from the previous lab. It now includes a `get_prob_dist()` function, which returns the probabilities of all tokens given the context.

Look over the implementation to be sure that you understand it.

In [ ]:
import pickle
BOS = '<BOS>'
EOS = '<EOS>'
OOV = '<OOV>'
class NGramLM:
    def __init__(self, path, smoothing=0.001, verbose=False):
        with open(path, 'rb') as fin:
            data = pickle.load(fin)
        self.n = data['n']
        self.V = set(data['V'])
        self.model = data['model']
        self.smoothing = smoothing
        self.verbose = verbose

    def get_prob_dist(self, context):
        # Take only the n-1 most recent context (Markov Assumption)
        context = tuple(context[-self.n+1:])
        # Add <BOS> tokens if the context is too short, i.e., it's at the start of the sequence
        while len(context) < (self.n-1):
            context = (BOS,) + context
        # Handle words that were not encountered during the training by replacing them with a special <OOV> token
        context = tuple((c if c in self.V else OOV) for c in context)
        if context in self.model:
            # Compute the probability distribution using a Maximum Likelihood Estimation and Laplace Smoothing
            norm = sum(self.model[context].values()) + self.smoothing * len(self.V)
            prob_dist = {k: (c + self.smoothing) / norm for k, c in self.model[context].items()}
            for word in self.V - prob_dist.keys():
                prob_dist[word] = self.smoothing / norm
        else:
            # Simplified formula if we never encountered this context; the probability of all tokens is uniform
            prob = 1 / len(self.V)
            prob_dist = {k: prob for k in self.V}
        prob_dist = dict(sorted(prob_dist.items(), key=lambda x: (-x[1], x[0])))
        return prob_dist

In [ ]:
# Load pre-built n-gram languae models
model_unigram = NGramLM('arthur-conan-doyle.tok.train.n1.pkl')
model_bigram = NGramLM('arthur-conan-doyle.tok.train.n2.pkl')
model_trigram = NGramLM('arthur-conan-doyle.tok.train.n3.pkl')
model_4gram = NGramLM('arthur-conan-doyle.tok.train.n4.pkl')
model_5gram = NGramLM('arthur-conan-doyle.tok.train.n5.pkl')

Let's take a look at some of the probability distributions.

Are they reasonable?

In [ ]:
model_bigram.get_prob_dist(['my'])

In [ ]:
model_bigram.get_prob_dist(['.'])

In [ ]:
model_trigram.get_prob_dist(['my', 'dear'])

Great, now we have all the tools we need to start generating text!

We'll start with a simple greedy generation approach. Your task is to implement greedy generation below.

Note: we have a `max_length` parameter to be sure that the generation process doesn't go on forever. You can stop when your sequence either reaches an `<EOS>` token or is the maximum length.

In [ ]:
from typing import List
def greedy_generation(model: NGramLM, context: List[str], max_length: int = 100) -> List[str]:
    pass
    # your code here

greedy_generation(model_trigram, ['""', 'My', 'dear', 'Watson'])

Great! How does the generation look? Feel free to try out a several samples below.

Is it deterministic? Are the generated sequences interesting?

Consider trying different model types. What are the different qualities that you see from the unigram, bigram, trigram, 4-gram, and 5-gram models?

In [ ]:
# your code here

Now it's time to implement sampling.

We now include a `topk` argument. This reduces the candidate set of probabilities down to only the `topk` highest-probability items. This helps reduce the chance of generating highly unlikely sequences.

Note: consider using [`random.choices`](https://docs.python.org/3/library/random.html#random.choices) to help in sampling.

In [ ]:
from typing import List
import random
def sampling_generation(model: NGramLM, context: List[str], max_length: int = 100, topk=10) -> List[str]:
    pass
    # your code here

sampling_generation(model_trigram, ['""', 'My', 'dear', 'Watson'])

Now qualitatively compare your sampling generation with the greedy generation.

What do you notice about the generated sequences? How do models of different sizes behave? What is the effect of `topk`?

In [ ]:
# your code here

## Optional Extras:
 - Try implementing a beam search strategy. Does it tend to lead to qualitatively better results than the other two approaches?
 - What strategy might you take to efficiently find the most likely sequence for an n-gram language model?